## Welcome to the Tutorial on Multiplexing Nanotextural Structures with Oneiros
In this notebook, we will learn how to use `Oneiros`, a part of the `NanTex` library, to generate multiplexed nanotextural structures from single-channel input data.

### Requirements
- Basic knowledge of Python
- Python 3.11 or higher
- Poetry

We assume you followed the installation instructions in the README.md file. If you haven't, please do so **before** proceeding.

## Part 0: Preparations

### Dependencies

In [ ]:
## Dependencies
from nantex.data_postprocessing.oneiros import Oneiros
from nantex.util import pyDialogue as pyD
from nantex.util import pltStyler
from glob import glob

import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

### Overview
``` Python
# Dreamer
# Dreamer = Oneiros(num_features = 3,               # Number of features in the dataset
#                   DEBUG=True,                     # Debug mode <- leave on for the first run
#                   data_paths_in = None,           # Path to the input data <- in MS Windows, a "None" will trigger a file dialog
#                   data_path_out = None,           # Path to the output data <- in MS Windows, a "None" will trigger a file dialog when you need to save the data
#                   mode = 'has_ground_truth')      # Mode of the data processing <- choose dependent on the data you are about to load in


# Dreamer WINODWS
# -> Dreamer.from_explorer() - this function will call the file explorer dialog to choose the input and output files

## There are two convenience functions that we will use in this notebook:
# - Dreamer.with_ground_truth() - this function will call "from_exploer" with the mode set to "has_ground_truth"
# - Dreamer.without_ground_truth() - this function will call "from_exploer" with the mode set to "no_ground_truth"

# You need to provide the number of present features and if you want to run it in DEBUG morde or not.
# In windows, you can conveniently choose multiple files using the file explorere dialog. 

# Dreamer LINUX
# -> Dreamer.from_glob() - this function will take a number of paths or glob patterns as input and load the data from the files
# Dreamer.from_glob(*['path/to/file1', 'path/to/file2', 'path/to/file3']) - this will load the data from the files
# You sill need to provide the number of present features and if you want to run it in DEBUG morde or not.
```

### Data
Oneiros can read both .npy and image (png, jpeg, jpg) files. If you want to call the Dreamer with ground truth, you need to provide the data as npy files. If you want to call the Dreamer without ground truth, you can provide the data as images. The npy should have the following shape (number_of_features + 1, y_size, x_size). The shape will match if you use the other packages provided within the NanTex project.

### Metaparameters
#### We will use the following metaparameters to run the Dreamer.
``` Json
"feature_static_threshodls" : {     # Static thresholds for each feature
    "feature_0": 0.1,               # Threshold for feature 0
    "feature_1": 0.1,               # Threshold for feature 1
    "feature_2": 0.1                # Threshold for feature 2 <- if you have more features, add them here
    },
"dynamic_thresholds": {             # Dynamic thresholds for each feature <- generates histogram and determins the main population of pixel values. 
                                    # For SMLM this usually is the background and some noise. Be careful using this with images that have a lot of signal.
    "auto_calculate": True,         # Automatically calculate dynamic thresholds using multiples of the standard deviation
    "upper": 3,                     # Upper threshold for dynamic threshold calculation <- hist argmax + 3 bins
    "lower": -2,                    # Lower threshold for dynamic threshold calculation <- hist argmax - 2 bins
    "std_factor": 2                 # Factor to multiply the standard deviation by < hist argmax +- std_factor * std
    },
"patch_size": (256, 256),           # Size of the patches to be extracted from the images
"dream_memory_shape" : None,        # Shape of the memory array for the DREAM algorithm <- will be autmatically populated
"patch_array_shape": None,          # Shape of the patch array <- will be automatically populated
"standardize": True,                # Standardize the patches
"normalize": False,                 # Normalize the patches <- not recommended
"tensortype": torch.float32,        # Type of tensor to be used
"out_type": np.uint8,               # Determine the type of the output tensor <- usually np.uint8
"weights_only": True,               # DEBUG parameter when loading a state_dict (torch) <- leave as it
"cast_all_to_img": True,            # Cast all tensors to images will normalize the output and cast it to the maximum range of the output type
"append_original_features": True,   # Append the original features to the output tensor <- for your convenience when exporting
"append_original_overlays": True,   # Append the original overlays to the output tensor
"append_dream_overlays": True,      # Append the dream overlays to the output tensor
"apply_static_thresholds": True,    # Apply the static thresholds to dream features 
"apply_dynamic_thresholds": True    # Apply the dynamic thresholds to dream features
```

#### The metaparameters are stored in a dictionary and can be accessed and changed using the following code:
``` Python
# Access the metaparameters
metaparameters = Dreamer.metaparameters

# Change the metaparameters
metaparameters['patch_size'] = (256, 256)

# Update the metaparameters
new_metaparameters = {
    'patch_size': (256, 256),
    'some_other_metaparameter': 'some_value'                    
                     }
Dreamer.metaparameters.update(new_metaparameters)
```


## Part 1: Multiplexing Data with Ground Truth

### UNIX: Select Data Files and Instantiate Oneiros

In [ ]:
# Select filepaths
path_to_data_directory: str = "path/to/your/data"
paths: list = glob(path_to_data_directory + "/*.npy")
paths = np.random.choice(paths, size=10, replace=False).tolist() # Select 10 random files for demonstration

# Select Pre-Trained Model State Dict
path_to_pretrained_model_state_dict: str = "path/to/your/pretrained/model.pt"

In [ ]:
# set up the dreamer for LINUX
Dreamer = Oneiros.from_glob(
    *paths, num_features=3, num_channels_out=3, DEBUG=True, VERBOSE = True, FREEMEMORY = False, mode="has_ground_truth"
)

In [ ]:
# change patchsize
Dreamer.metadata["patch_size"] = (256, 256)
Dreamer.metadata["normalize"] = False
Dreamer.metadata["standardize"] = True

pprint(Dreamer.metadata)

### UNIX: Multiplexing Data with Ground Truth

In [ ]:
# load the model
Dreamer.jumpstart_model(path_to_pretrained_model_state_dict)

If you want to load in more data without reinitializing the model, run:

``` Python
Dreamer.load_new_data_linux("path/to/new/data_1.npy", "path/to/new/data_2.npy", ...)
```

In [ ]:
Dreamer.dream()  # <- this will start the data processing

### UNIX: Checkpoint I - Multiplexing Data with Ground Truth

In [ ]:
# Visualize the first dream

# stylesheet
pltStyler().enforce_stylesheet()

Dreamer.visualize(
    dream_no=5,  # <- dream number to visualize
    cmap="inferno",  # <- colormap to use, defaults to 'gray
    ticks=False,  # <- on/off ticks
    return_fig_axs=False,
)  # <- return fig and axs objects

In [ ]:
# Export your results
# npy will export each dream to a 4D tensor
# png will export each dream to a folder of pngs

Dreamer.export(
    out_type="png",  # <- output type, choose from 'png' or 'npy' or implement your own
    outpath=".",
)  # <- output path

### WINDOWS: Load Data with Ground Truth via File Explorer

In [ ]:
# Select Pre-Trained Model State Dict
path_to_pretrained_model_state_dict: str = "path/to/your/pretrained/model_state_dict.pt"

In [ ]:
# set up the dreamer with random selection of data with ground truth
Dreamer = Oneiros.random_selection_with_ground_truth(
    num_selection=10, num_features=3, num_channels_out=3, DEBUG=True, VERBOSE = True, FREEMEMORY = False
)

In [ ]:
# change patchsize
Dreamer.metadata["patch_size"] = (256, 256)
Dreamer.metadata["normalize"] = False
Dreamer.metadata["standardize"] = True

pprint(Dreamer.metadata)

If you want to run the entire selected data without rabdom selection run:
``` Python
Dreamer = Oneiros.with_ground_truth(num_selection=5, num_features=3, num_channels_out=3, DEBUG=True, VERBOSE = True, FREEMEMORY = False)
```

### WINDOWS: Multiplexing Data with Ground Truth

In [ ]:
# Quickstart model
# Just choose the torch checkpoint file and the model will be loaded
Dreamer.jumpstart_model(path_to_pretrained_model_state_dict)

Id you want to load new data without reloading the model, run:
``` Python
Dreamer.load_new_data_windows()
```

In [ ]:
Dreamer.dream()

### WINDOWS: Checkpoint I - Multiplexing Data with Ground Truth

In [ ]:
# Visualize the first dream

# stylesheet
pltStyler().enforce_stylesheet()

Dreamer.visualize(
    dream_no=0,  # <- dream number to visualize
    cmap="inferno",  # <- colormap to use, defaults to 'gray
    ticks=False,  # <- on/off ticks
    return_fig_axs=False,
)  # <- return fig and axs objects

In [ ]:
## Export the results
Dreamer.export(
    out_type="single_npy"
)  # png <- image, npy <- numpy array, if you want more, you need to implement it

## Part II: Custom Postprocessing

In [ ]:
## Play with the static thresholds to delete background noise
Dreamer.metadata["feature_static_thresholds"].update(
    {
        "feature_0": 0.0,
        "feature_1": 0.0,
        "feature_2": 0.0,
    }
)
# Play with the dynamic thresholds to  delete structural noise
Dreamer.metadata["dynamic_thresholds"].update(
    {"auto_calculate": True, "upper": 3, "lower": -2, "std_factor": 3}
)


# Play with the dreamer parameters
Dreamer.metadata["apply_dynamic_thresholds"] = False
Dreamer.metadata["apply_static_thresholds"] = False

# choose what extras to export
Dreamer.metadata["append_dream_overlays"] = True
Dreamer.metadata["append_original_overlays"] = True
Dreamer.metadata["append_original_features"] = True

Dreamer.metadata["cast_all_to_img"] = (
    True  # if you dont want to export straight to image
)

# however, you have to re-run the post-processing
Dreamer.__post_process_data__()

In [ ]:
## Plot and vsualize the results
pltStyler().enforce_stylesheet()
Dreamer.visualize(dream_no=0, return_fig_axs=False)

In [ ]:
## Export the results
Dreamer.export(
    out_type="png"
)  # png <- image, npy <- numpy array, if you want m0re, you need to implement it

## Without Ground Truth

In [ ]:
# set up the dreamer
Dreamer = Oneiros.without_ground_truth(num_features=2, num_channels_out=2, DEBUG=True)

In [ ]:
Dreamer.jumpstart_model()

In [ ]:
Dreamer.load_new_data_windows()

In [ ]:
Dreamer.dream()

In [ ]:
# Visualize the first dream
pltStyler().enforce_stylesheet()
Dreamer.visualize(0, return_fig_axs=False, cmap="inferno")

In [ ]:
## Export the results
Dreamer.export(
    out_type="png"
)  # png <- image, npy <- numpy array, if you want mire, you need to implement it